In [1]:
from flask import Flask, render_template, request, url_for, send_file, jsonify
from selenium import webdriver
from bs4 import BeautifulSoup

import os
import requests
from collections import defaultdict
import json

from utils.extract import *
from utils.create_pdf import *
from readability import Document

from utils.create_pdf.create_article import *

In [2]:
dictWord = eval(open('utils/data/autoFindPattern/wordPG.txt', 'r').read())
phraseV = eval(open('utils/data/autoFindPattern/phrase(V).txt', 'r').read())

# read translation
TRANS = eval(open('utils/data/final TRANS.txt', 'r').read()) # tran[pos][word] = [translation...]

In [3]:
app = Flask(__name__ )
import datetime
# egp = load_egp() # grammar pattern

if not os.path.exists('download'):
    os.makedirs('download')

@app.route('/', methods=['POST', 'GET'])
def index():
    return render_template('index.html')
    #return render_template('format.html', title=title, publish_date=publish_date, content=new, user_level=user_level, grade=grade)

@app.route('/handle_data', methods=['POST', 'GET'])
def handle_data():
    user_level = request.form['user_level']
    try:
        url = request.form['url']
        response = requests.get(url)
        doc = Document(remove_a(response.text))
        title = doc.short_title()
        publish_date = getPublishDate(url)
        content = doc.summary()
        type_ = 'url'
    except:
        url = ''
        text = request.form['text']
        title = ''
        content = text
        type_ = 'text'
        publish_date = ''
        
    content = clean_content(content, type_)
    new = create_article(title, user_level, content, \
                         set(dictWord['V'].keys()), set(dictWord['N'].keys()), set(dictWord['ADJ'].keys()))
    
    return render_template('format.html', title=title, publish_date=publish_date, \
                           user_level=user_level, content=new) 

@app.route('/download/<filename>', methods=['GET'])
def return_reformatted(filename):
    try:
        return send_file('download/'+filename)# , as_attachment=True
    except Exception as e:
        return str(e)

@app.route('/ajax', methods = ['POST'])
def ajax_request():
    word = request.form['word'].lower() if request.form['pos'] != 'x' else request.form['word'].split()[0].lower()  
    poses = ['V', 'N', 'ADJ'] if len(request.form['word'].split())==1 else [p.upper() for p in request.form['word'].split()[1:]]
    
    # patternTable[pos] = [(pat, colls, (en, ch, source)), ...] 
    patternTable = defaultdict(lambda: [])
    # phraseTable[pos][phrase] = [pat, (colls, (en, ch, source)), ...] 
    phraseTable = defaultdict(lambda: defaultdict(lambda: []))
    # phraseOrder = [phrase...]
    phraseOrder = []
    # trans[type][pos] = [translation]
    trans = defaultdict(lambda: defaultdict(lambda: list())) 
    
    for pos in poses:
        if word in dictWord[pos].keys():
            # TODO須處理個數，以後可能動態
            for pat, colls, examp in dictWord[pos][word][:5]:
                patternTable[pos] += [(pat, ', '.join(colls[:3]), examp)]

        if pos == 'V' and word in phraseV.keys():
            # 前面以過濾過phrase至多3個, pat已用std過濾
            phraseOrder = sorted(phraseV[word].keys(), key=lambda x: -int(x.rsplit('%', 1)[1]))
            for phrase in phraseOrder:
                for pat, colls, examp in phraseV[word][phrase]:
                    phraseTable[pos][phrase] += [(pat, ', '.join(colls[:3]), examp)]
                    phrase = phrase.split('%')[0]
                    if phrase in TRANS['phrase'][pos].keys():
                        trans['phrase'][phrase] = TRANS['phrase'][pos][phrase]
                    else:
                        trans['phrase'][phrase] = []

        if word in set(TRANS['pat'][pos].keys()):
            trans['pat'][pos] = TRANS['pat'][pos][word]
    return jsonify(patternTable=patternTable, \
                   phraseTable=phraseTable, phraseOrder=phraseOrder, \
                   trans=trans)

#static url cache buster
@app.context_processor
def override_url_for():
    return dict(url_for=dated_url_for)

def dated_url_for(endpoint, **values):
    if endpoint == 'static':
        filename = values.get('filename', None)
        if filename:
            file_path = os.path.join(app.root_path,
                                     endpoint, filename)
            values['q'] = int(os.stat(file_path).st_mtime)
    return url_for(endpoint, **values)   

if __name__ == '__main__':
    app.run(debug=False)
    #app.run(host='0.0.0.0', port=int("5487"), debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jul/2018 14:45:12] "GET / HTTP/1.1" 200 -


She is 20 and finishing a degree in psychology at a Delhi university. She wears Zara skinny jeans and H&M T-shirts and hangs out with her girlfriends at one of Delhi's myriad American-style malls and coffee shops.

She listens to R&B and EDM on Apple Music; her favorite song is "She Will Be Loved" by Maroon 5. Cable TV is so yesterday; she streams shows and movies on Netflix. Her favorite? "Something Borrowed," in which a young woman falls in love with her best friend's fiancé.

But when it comes to marriage, Naina's views might shock American women her age. She reflects a way of thinking long engrained in the culture of my homeland: Your parents know best.
[['p', ["She is 20 and finishing a degree in psychology at a Delhi university. She wears Zara skinny jeans and H&M T-shirts and hangs out with her girlfriends at one of Delhi's myriad American-style malls and coffee shops."]], ['p', ["She is 20 and finishing a degree in psychology at a Delhi university. She wears Zara skinny jeans a

127.0.0.1 - - [17/Jul/2018 14:45:15] "POST /handle_data HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:17] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:18] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:18] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:19] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:19] "GET /static/img/adj_burned.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Jul/2018 14:45:20] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:21] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:21] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:21] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:22] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:22] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:32] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:32] "POST /ajax HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 14:45:33] "POST /ajax HTTP/1.1" 200 